### **Notebook objective**: create a rating per team and then have a running mean for each team for season 22/23 <br> - 6 rates per team : passing, tackles, rushing, sacks, receiving, interceptions <br> - Running mean : the running mean for the last 5 games will define the rating features for the prediction model

In [18]:
import pandas as pd
import numpy as np
import requests
import os
import json

pd.options.mode.chained_assignment = None  # default='warn'

df = pd.read_json("espn_rosters (1).json")

#### DATA CLEANING

In [19]:
#Replace " " in columns' name by "_"
df.columns = df.columns.str.replace(' ','_')
df.head()

,game_id,team,player_name,player_url,pass_completion,pass_yds,pass_avg,pass_td,pass_int,sacks,...,kicking_pct,kicking_long,kicking_xp,kicking_pts,punting_no,punting_yds,punting_avg,punting_tb,punting_in_20,punting_long
0,401220131,Dolphins,Zach Sieler,https://www.espn.com/nfl/player/_/id/3057956/z...,None,NaN,NaN,NaN,NaN,None,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,401220131,Dolphins,Patrick Laird,https://www.espn.com/nfl/player/_/id/3127211/p...,None,NaN,NaN,NaN,NaN,None,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,401220313,Seahawks,Chris Carson,https://www.espn.com/nfl/player/_/id/3919596/c...,None,NaN,NaN,NaN,NaN,None,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,401220131,Dolphins,Jerome Baker,https://www.espn.com/nfl/player/_/id/3915507/j...,None,NaN,NaN,NaN,NaN,None,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,401220225,Texans,Randall Cobb,https://www.espn.com/nfl/player/_/id/14053/ran...,None,NaN,NaN,NaN,NaN,None,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Missing values

In [20]:
#Verification of the columns with no missing value
clean_columns = []
for column in df.columns:
    if 100*df[column].isnull().sum()/df.shape[0] == 0.000000 :
        clean_columns.append(column)
display(clean_columns)

#Replace all the missing value with 0 because the NaN means the metrics is not relevent for the concerned player
df = df.fillna(0)

#Check of the percentage of missing values after cleaning
display(100*df.isnull().sum()/df.shape[0])

['game_id', 'team', 'player_name', 'player_url']

game_id              0.0
team                 0.0
player_name          0.0
player_url           0.0
pass_completion      0.0
pass_yds             0.0
pass_avg             0.0
pass_td              0.0
pass_int             0.0
sacks                0.0
qbr                  0.0
rtg                  0.0
rush_car             0.0
rush_yds             0.0
rush_avg             0.0
rush_td              0.0
rush_long            0.0
receptions           0.0
rec_yds              0.0
rec_avg              0.0
rec_td               0.0
rec_long             0.0
rec_tgs              0.0
fumbles              0.0
fumbles_lost         0.0
fumbles_rec          0.0
defense_tot          0.0
defense_solo         0.0
defense_sacks        0.0
defense_tfl          0.0
defense_pd           0.0
defense_qb_hits      0.0
defense_td           0.0
interceptions        0.0
interceptions_yds    0.0
interceptions_td     0.0
kicks_return_no      0.0
kicks_return_yds     0.0
kicks_return_avg     0.0
kicks_return_long    0.0


##### Columns' types

In [21]:
#Pass completion: transform "x/y" (str) into a ratio (float)
df['pc_1'] = df['pass_completion'].apply(lambda x : x.split('/')[0] if x!=0 else x)
df['pc_2'] = df['pass_completion'].apply(lambda x : x.split('/')[1] if x!=0 else x)

#Convert str to float in pc_1 and pc_2 before looping
df['pc_1'] = df['pc_1'].astype(float)
df['pc_2'] = df['pc_2'].astype(float)

for i in range(len(df)):
    if df['pc_2'].iloc[i] != 0 :
        df['pass_completion'].iloc[i] = df['pc_1'].iloc[i] / df['pc_2'].iloc[i]
    else: df['pass_completion'].iloc[i] = 0

#Sacks : transform "x-y" (str) into a ratio (float)
df['sacks_1'] = df['sacks'].apply(lambda x : x.split('-')[0] if x!=0 else x)
df['sacks_2'] = df['sacks'].apply(lambda x : x.split('-')[1] if x!=0 else x)

#Convert str to float in sacks_1 and sacks_2 before looping
df['sacks_1'] = df['sacks_1'].astype(float)
df['sacks_2'] = df['sacks_2'].astype(float)

for i in range(len(df)):
    if df['sacks_2'].iloc[i] != 0:
        df['sacks'].iloc[i] = df['sacks_1'].iloc[i] / df['sacks_2'].iloc[i]
    else: df['sacks'].iloc[i] = 0

In [22]:
df['pass_completion']=df.pass_completion.apply(lambda x : float(x))
df['sacks']=df.sacks.apply(lambda x : float(x))

In [23]:
df['qbr']=df.sacks.apply(lambda x : float(x))

In [24]:
#Check of the global datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79606 entries, 0 to 79605
Data columns (total 61 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   game_id            79606 non-null  int64  
 1   team               79606 non-null  object 
 2   player_name        79606 non-null  object 
 3   player_url         79606 non-null  object 
 4   pass_completion    79606 non-null  float64
 5   pass_yds           79606 non-null  float64
 6   pass_avg           79606 non-null  float64
 7   pass_td            79606 non-null  float64
 8   pass_int           79606 non-null  float64
 9   sacks              79606 non-null  float64
 10  qbr                79606 non-null  float64
 11  rtg                79606 non-null  float64
 12  rush_car           79606 non-null  float64
 13  rush_yds           79606 non-null  float64
 14  rush_avg           79606 non-null  float64
 15  rush_td            79606 non-null  float64
 16  rush_long          796

In [25]:
#Drop useless preprocessing columns (pc_1, pc_2, sacks_1, sacks_2)
df = df.drop(columns=['pc_1', 'pc_2', 'sacks_1', 'sacks_2'], axis=1)

##### Total metrics by player

In [26]:
#Create a new column with the sum of all available metrics per player
metrics = ['pass_completion',
       'pass_yds', 'pass_avg', 'pass_td', 'pass_int', 'sacks', 'qbr', 'rtg',
       'rush_car', 'rush_yds', 'rush_avg', 'rush_td', 'rush_long',
       'receptions', 'rec_yds', 'rec_avg', 'rec_td', 'rec_long', 'rec_tgs',
       'fumbles', 'fumbles_lost', 'fumbles_rec', 'defense_tot', 'defense_solo',
       'defense_sacks', 'defense_tfl', 'defense_pd', 'defense_qb_hits',
       'defense_td', 'interceptions', 'interceptions_yds', 'interceptions_td',
       'kicks_return_no', 'kicks_return_yds', 'kicks_return_avg',
       'kicks_return_long', 'kicks_return_td', 'punt_return_no',
       'punt_return_yds', 'punt_return_avg', 'punt_return_long',
       'punt_return_td', 'kicking_fg', 'kicking_pct', 'kicking_long',
       'kicking_xp', 'kicking_pts', 'punting_no', 'punting_yds', 'punting_avg',
       'punting_tb', 'punting_in_20', 'punting_long']

df['rating_player'] = df[metrics].sum(axis=1)
df.head()

C:\Users\alice\AppData\Local\Temp\ipykernel_6084\3987183445.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['rating_player'] = df[metrics].sum(axis=1)


,game_id,team,player_name,player_url,pass_completion,pass_yds,pass_avg,pass_td,pass_int,sacks,...,kicking_long,kicking_xp,kicking_pts,punting_no,punting_yds,punting_avg,punting_tb,punting_in_20,punting_long,rating_player
0,401220131,Dolphins,Zach Sieler,https://www.espn.com/nfl/player/_/id/3057956/z...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,401220131,Dolphins,Patrick Laird,https://www.espn.com/nfl/player/_/id/3127211/p...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.5
2,401220313,Seahawks,Chris Carson,https://www.espn.com/nfl/player/_/id/3919596/c...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0
3,401220131,Dolphins,Jerome Baker,https://www.espn.com/nfl/player/_/id/3915507/j...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0
4,401220225,Texans,Randall Cobb,https://www.espn.com/nfl/player/_/id/14053/ran...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.5


##### RATING FUNCTION

#### First thing first: a rating taking into account all the metrics

In [27]:
#Define a function to calculate a rating per team
def calculate_rating_team(idgame, team):
    df_game=df[df['game_id'] == idgame]
    df_team=df_game[df_game['team'] == team]
    rating_team = df_team['rating_player'].sum()
    return team, rating_team

###### Test of the function with one game id

In [28]:
test1 = df[df['game_id'] == 401220131]
test1['team'].unique()

array(['Dolphins', 'Patriots'], dtype=object)

In [29]:
calculate_rating_team(401220131, 'Dolphins')

('Dolphins', 1364.9888888888888)

In [30]:
calculate_rating_team(401220131, 'Patriots')

('Patriots', 1346.5561403508773)

###### Looks like it is working, youpi ! Let's industrialize now !

#### A rating per metric

In [31]:
len(metrics) # The number of rating we will have at the end

53

##### DATASET CREATION

In [32]:
# Import main dataset
df_scores = pd.read_csv("scores_prep.csv")
df_scores.head()

,Unnamed: 0,season,week,awayteam,hometeam,awayscore,homescore,idgame,score_abs,winner_home,winner_away
0,0,2005,1,Raiders,Patriots,20,30,250908017,10,1,0
1,1,2005,1,Texans,Bills,7,22,250911002,15,1,0
2,2,2005,1,Bengals,Browns,27,13,250911005,-14,0,1
3,3,2005,1,Jets,Chiefs,7,27,250911012,20,1,0
4,4,2005,1,Broncos,Dolphins,10,34,250911015,24,1,0


In [33]:
drop_columns = ['Unnamed: 0']
df_scores = df_scores.drop(drop_columns, axis=1)

In [34]:
# Create two new columns for the rating by team
df_scores['rating_team_away'] = 0
df_scores['rating_team_home'] = 0

In [35]:
df_scores.head()

,season,week,awayteam,hometeam,awayscore,homescore,idgame,score_abs,winner_home,winner_away,rating_team_away,rating_team_home
0,2005,1,Raiders,Patriots,20,30,250908017,10,1,0,0,0
1,2005,1,Texans,Bills,7,22,250911002,15,1,0,0,0
2,2005,1,Bengals,Browns,27,13,250911005,-14,0,1,0,0
3,2005,1,Jets,Chiefs,7,27,250911012,20,1,0,0,0
4,2005,1,Broncos,Dolphins,10,34,250911015,24,1,0,0,0


In [ ]:
# Fill the column rating team away
list_team_1 = []
for game in df_feature['game_id']:
    list_team_1.append(game_team_1(game))
list_team_1

In [ ]:
#Define a function to calculate a rating per team
def calculate_rating_awayteam(idgame):
    df_game=df_scores[df_scores['idgame'] == idgame]
    df_team=df_game[df_game['awayteam'] == team]
    rating_team = df_team['rating_player'].sum()
    return team, rating_team

#### Start testing

In [37]:
df_rating = df.groupby(['game_id', 'team'])['rating_player'].sum()
df_rating.head()

game_id    team      
400950241  Dolphins      1302.036508
           Saints        1712.893031
400951552  Buccaneers    1853.631884
           Patriots      1891.278571
400951553  Cardinals     1688.845932
Name: rating_player, dtype: float64

In [38]:
type(df_rating)

pandas.core.series.Series

In [39]:
df_rating.shape

(2584,)

In [40]:
df_rating.to_frame()

rating_player
game_id   team                     
400950241 Dolphins      1302.036508
          Saints        1712.893031
400951552 Buccaneers    1853.631884
          Patriots      1891.278571
400951553 Cardinals     1688.845932
...                             ...
401326598 Chiefs        1619.346970
401326599 49ers         2199.549519
          Rams          1745.326520
401326600 Chargers      1960.745536
          Raiders       1861.688889

[2584 rows x 1 columns]

In [41]:
df_rating.info()

<class 'pandas.core.series.Series'>
MultiIndex: 2584 entries, (400950241, 'Dolphins') to (401326600, 'Raiders')
Series name: rating_player
Non-Null Count  Dtype  
--------------  -----  
2584 non-null   float64
dtypes: float64(1)
memory usage: 38.2+ KB


In [42]:
df_rating.reset_index()

,game_id,team,rating_player
0,400950241,Dolphins,1302.036508
1,400950241,Saints,1712.893031
2,400951552,Buccaneers,1853.631884
3,400951552,Patriots,1891.278571
4,400951553,Cardinals,1688.845932
...,...,...,...
2579,401326598,Chiefs,1619.346970
2580,401326599,49ers,2199.549519
2581,401326599,Rams,1745.326520
2582,401326600,Chargers,1960.745536


In [43]:
df_rating_global = df.groupby(['game_id', 'team'])[metrics].sum()
df_rating_global.reset_index()

,game_id,team,pass_completion,pass_yds,pass_avg,pass_td,pass_int,sacks,qbr,rtg,...,punt_return_td,kicking_pct,kicking_long,kicking_pts,punting_no,punting_yds,punting_avg,punting_tb,punting_in_20,punting_long
0,400950241,Dolphins,0.714286,164.0,5.9,0.0,1.0,0.111111,0.111111,71.1,...,0.0,0.0,0.0,0.0,6.0,263.0,43.8,0.0,2.0,49.0
1,400950241,Saints,0.707317,268.0,6.5,2.0,0.0,0.142857,0.142857,104.5,...,0.0,66.7,45.0,8.0,3.0,141.0,47.0,0.0,0.0,58.0
2,400951552,Buccaneers,0.565217,334.0,7.3,1.0,0.0,0.133333,0.133333,86.7,...,0.0,0.0,0.0,2.0,6.0,271.0,45.2,0.0,3.0,57.0
3,400951552,Patriots,0.750000,303.0,7.6,1.0,1.0,0.214286,0.214286,94.1,...,0.0,100.0,48.0,13.0,4.0,188.0,47.0,0.0,1.0,55.0
4,400951553,Cardinals,0.510638,273.0,5.8,1.0,0.0,0.117647,0.117647,75.9,...,0.0,100.0,43.0,4.0,5.0,230.0,46.0,1.0,2.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2579,401326598,Chiefs,0.613636,270.0,6.1,2.0,0.0,0.066667,0.066667,93.9,...,0.0,100.0,51.0,8.0,2.0,101.0,50.5,2.0,0.0,58.0
2580,401326599,49ers,1.718750,340.0,33.9,2.0,2.0,0.115385,0.115385,245.8,...,0.0,100.0,42.0,9.0,4.0,175.0,87.5,0.0,1.0,90.0
2581,401326599,Rams,0.656250,238.0,7.4,3.0,2.0,0.135135,0.135135,93.0,...,0.0,100.0,43.0,6.0,4.0,198.0,49.5,0.0,2.0,59.0
2582,401326600,Chargers,0.531250,383.0,6.0,3.0,1.0,0.107143,0.107143,80.4,...,0.0,50.0,41.0,6.0,3.0,162.0,54.0,0.0,0.0,56.0


In [44]:
type(df_rating_global)

pandas.core.frame.DataFrame

## The dataframe to use for moving average is : df_rating_global

In [45]:
# Creation of the csv
df_rating_global.to_csv('NFL_team_rating.csv')